# ДЗ к Уроку 3 "Логистическая регрессия. Log Loss"

### Задание 1: Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.

In [7]:
import numpy as np
import matplotlib.pyplot as plt

In [8]:
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 1, 3, 0, 5, 10, 1, 2],
              [500, 700, 750, 600, 1450,
               800, 1500, 2000, 450, 1000],
              [1, 1, 2, 1, 2, 
               1, 3, 3, 1, 2]], dtype = np.float64)

y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype = np.float64)

In [9]:
def calc_std_feat(x):
    res = (x - x.mean()) / x.std()
    return res

In [10]:
X_st = X.copy()
X_st[2, :] = calc_std_feat(X[2, :])

In [11]:
X_st

array([[ 1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ,  1.        ,  1.        ],
       [ 1.        ,  1.        ,  2.        ,  1.        ,  3.        ,
         0.        ,  5.        , 10.        ,  1.        ,  2.        ],
       [-0.97958969, -0.56713087, -0.46401617, -0.77336028,  0.97958969,
        -0.36090146,  1.08270439,  2.11385144, -1.08270439,  0.05155735],
       [ 1.        ,  1.        ,  2.        ,  1.        ,  2.        ,
         1.        ,  3.        ,  3.        ,  1.        ,  2.        ]])

In [69]:
def calc_logloss(y, y_pred):
    const = 1e-5
    y_pred = y_pred.copy()
    y_pred = np.clip(y_pred, a_min=const, a_max=1-const)
    err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
    return err

In [75]:
# Пример применения
y1 = np.array([1, 0])
y_pred1 = np.array([0.9, 0.1])
calc_logloss(y1, y_pred1)

0.10536051565782628

In [76]:
def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

In [77]:
### Logistic Regression
def eval_model(X, y, iterations, alpha=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        W -= alpha * (1/n * np.dot((y_pred - y), X.T))
    if i % (iterations / 10) == 0:
        print(i, W, err)
    return W, err

In [78]:
W = eval_model(X_st, y, iterations=1000, alpha=1e-5)

1000 [ 0.49282757 -0.15007512  0.64748969  1.51727928] 1.201313380901165


### Задание 2: Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.


In [17]:
import pandas as pd

In [108]:
alphas = [0.00001, 0.0001, 0.001, 0.01, 0.1]
iterations = range(1, 10001, 1000)
errors_df = pd.DataFrame([], columns=['alpha', 'iteration', 'err'])
for a in alphas:
    for n_iter in iterations:
        _, err = eval_model(X_st,y, n_iter, a)
        errors_df.loc[len(errors_df)] = [a, n_iter, err]

In [109]:
best_err = errors_df.err.min()
errors_df.query('err == @best_err')

,alpha,iteration,err
49,0.1,9001.0,0.258656


### Задание 3: Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba)

In [114]:
def calc_pred_proba(W,X):
    y_pred_proba = 1/(1+np.exp(-np.dot(W,X))) 
    return y_pred_proba

In [116]:
W, err = eval_model(X_st,y, 9000, 0.1)

9000 [-10.67239108  -1.40571631  -2.1950503    9.01453862] 0.2586628760279488


In [117]:
calc_pred_proba(W, X_st)

array([0.28632092, 0.13959138, 0.99618044, 0.20326622, 0.72894738,
       0.29615909, 0.99905752, 0.08899659, 0.33470563, 0.98825   ])

### Задание 4: Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).

In [123]:
def calc_pred(W,X):
    probabilities = calc_pred_proba(W, X)
    y_pred = probabilities.round(0)
    return y_pred

In [131]:
y_pred = calc_pred(W, X_st)
y_pred

array([0., 0., 1., 0., 1., 0., 1., 0., 0., 1.])

### Задание 5: Посчитайте Accuracy, матрицу ошибок, точность и полноту, а также F1 score.

In [133]:
def accuracy(y_true, y_pred):
    acc = (y_pred == y_true).mean()
    return acc

In [134]:
accuracy(y, y_pred)

0.9

In [162]:
def class_error(y_true, y_pred):
    TP = (y_pred*y).sum()
    TN = ((y_pred+y)==0).sum()
    FN = ((y*y_pred) != y).sum()
    FP = ((y*y_pred) != y_pred).sum()
    return TP, TN, FN, FP

In [163]:
def error_matrix(y_true, y_pred):
    TP,TN,FN,FP = class_error(y_true, y_pred)
    matrix = pd.DataFrame([[TP,FP],[FN,TN]], columns=['class1', 'class0'], index=['class1','class0'])
    return matrix

In [164]:
error_matrix(y, y_pred)

,class1,class0
class1,4.0,0
class0,1.0,5


In [165]:
def precision(y_true, y_pred):
    TP,_,_,FP = class_error(y_true, y_pred)
    return TP/(TP+FP)

In [166]:
precision(y, y_pred)

1.0

In [167]:
def recall(y_true, y_pred):
    TP,_,FN,_ = class_error(y_true, y_pred)
    return TP/(TP+FN)

In [168]:
recall(y, y_pred)

0.8

In [169]:
def f1score(y_true, y_pred):
    p = precision(y, y_pred)    
    r = recall(y, y_pred)    
    return(2*p*r)/(p+r)

In [170]:
f1score(y, y_pred)

0.888888888888889

### Задание 6: Могла ли модель переобучиться? Почему?

Да, могла, т.к. переобучение появляется вследствие минимизации эмпирического риска (ошибки на обучающей выборке). Чтобы этого избежать, можно использовать регуляризацию 